In [1]:
import cv2
from neuralnet import NN, ConvLayer
from neuralnet import Pooling
from neuralnet import Flatten
from neuralnet import Dense
from neuralnet import load_model
import numpy as np
from neuralnet import Trainer
from neuralnet import Preprocess

In [2]:
image = cv2.imread("251.jpeg")
model = NN(image.shape)
model.add(
    ConvLayer(
        input_shape=image.shape,
        padding=0,
        num_filters=1,
        filter_size=(3, 3),
        stride=1,
        detector_function="relu",
    )
)
model.add(Pooling(mode="max", pool_size=(2, 2), stride=2))
model.add(Flatten())

flat_shape = model.layers[2].feature_map_shape
model.add(Dense(8, flat_shape, "relu"))

model.add(Dense(1, 8, "sigmoid"))
model.save_model("base.json", 4)

In [3]:
data_preprocess = Preprocess("../data", False)
train, test = data_preprocess.get_data_split(split=(0.9,0.1), max_data_len=100)

print(train.get_images().shape)
print(test.get_images().shape)

(90, 256, 256, 3)
(10, 256, 256, 3)


In [4]:
trainer = Trainer(
    model,
    1,
    0.1,
    train.get_images(),
    train.get_labels(),
    test_input=test.get_images(),
    test_label=test.get_labels(),
)
trainer.fit()

Training model...
Epoch 1/1


c:\DATA\Smt9\mll\tubes-cnn\src\neuralnet\Activation.py:49: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-input))


Confusion Matrix:
[[ 0 42]
 [ 0 48]]
Accuracy: 53.33%
Average Precision: 26.67%
Average Recall: 50.00%
Average F1 Score: 34.78%
Training finished. Saving model...


In [6]:
# take 10 data from test set and predict
test_images = test.get_images()[:10]
test_labels = test.get_labels()[:10]
pred = model.predict(test_images)
id2label = {0:"bear" , 1 :"panda"}

for i in range(len(pred)):
    print("Predicted: ", id2label[pred[i]], "Actual: ", id2label[test_labels[i][0]])

Predicted:  panda Actual:  panda
Predicted:  panda Actual:  panda
Predicted:  panda Actual:  panda
Predicted:  panda Actual:  panda
Predicted:  panda Actual:  bear
Predicted:  panda Actual:  bear
Predicted:  panda Actual:  panda
Predicted:  panda Actual:  bear
Predicted:  panda Actual:  bear
Predicted:  panda Actual:  bear
